<h1 align="center"><font color="yellow">LangChain 1: Modelos de Prompt para GPT-3.5 e outros LLMs (código aberto)</font></h1>

<font color="yellow">Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro</font>

# Contextualizando

Seguindo o estudo do `LangChain`, exploraremos `Templates de Prompt`, `Few-Shot Prompt Templates` e seletores de exemplo. Esses são os principais recursos do LangChain que oferecem `suporte à engenharia de Prompt para LLMs`, como as alternativas de sistema operacional `GPT 3 da OpenAI`, `Cohere` e `Hugging Face`. `LangChain` é um Framework popular que permite aos usuários criar rapidamente `aplicativos` e `pipelines` em torno de `Large Language Models`. Ele se integra diretamente aos modelos `GPT-3` e `GPT-3.5` da OpenAI e às alternativas de código aberto do `Hugging Face`, como os modelos `flan-t5 do Google`. 

Ele pode ser usado para `ChatBots`, `perguntas-respostas generativas` (GQA), `resumos` e muito mais. A ideia central da biblioteca é que podemos `"encadear"` diferentes componentes para criar casos de uso mais avançados em torno de LLMs. As cadeias podem consistir em vários componentes de vários módulos. 

# Engenharia de Prompt

<font color="orange">Bora aprender os fundamentos da Engenharia de Prompt:</font>

In [ ]:
%pip install langchain openai

# Estrutura de um prompt

Um prompt pode consistir em vários componentes:

* Instruções

* Informação externa ou contexto

* Entrada ou consulta do usuário

* Indicador de saída

Nem todos os prompts requerem todos esses componentes, mas geralmente um bom `prompt` usará dois ou mais deles. Vamos definir o que todos eles são com mais precisão.

As `Instruções` dizem ao modelo o que fazer, normalmente como ele deve usar entradas e/ou informações externas para produzir a saída que queremos.

`Informações externas ou contexto` são informações adicionais que inserimos manualmente no prompt, recuperamos por meio de um banco de dados vetorial (`memória de longo prazo`) ou extraímos por outros meios (`chamadas de API`, `cálculos`, etc.).

A `Entrada ou consulta do usuário` é normalmente uma consulta inserida diretamente pelo usuário do sistema.

O `Indicador de saída` é o início do texto gerado. Para um modelo que gera código Python, podemos colocar `import` (já que a maioria dos scripts Python começa com uma importação de biblioteca) ou um `chatbot` pode começar com `Chatbot:` (supondo que formatemos o script do chatbot como linhas de texto intercambiáveis entre o usuário e o chatbot).

<font color="yellow">Cada um desses componentes geralmente deve ser colocado na ordem em que os descrevemos.</font> Começamos com `Instruções`, fornecemos `contexto` (se necessário), depois adicionamos a `entrada do usuário` e, finalmente, terminamos com o `indicador de saída`.

In [1]:
prompt = """Responda a pergunta com base no contexto abaixo. Se a pergunta não puder ser respondida
            usando as informações fornecidas, responda com "Não sei".

Context: Os Large Language Models (LLMs) são os modelos mais recentes usados em NLP. Seu desempenho
         superior em relação a modelos menores os tornou incrivelmente úteis para desenvolvedores que
         constroem aplicativos habilitados para NLP. Esses modelos podem ser acessados através da
         biblioteca `Transformers` do Hugging Face, via OpenAI usando a biblioteca `openai` e via
         Cohere usando a biblioteca `cohere`.

Question: Quais bibliotecas e provedores de modelos oferecem LLMs?

Answer: """

Neste exemplo temos:

`Instruções`

`Contexto`

Pergunta (`Entrada do usuário`)

`Indicador de saída` ("Answer: ")


Vamos tentar enviar isso para um modelo `GPT-3`. Usaremos a biblioteca `LangChain`, mas você também pode usar a biblioteca `openai` diretamente. Em ambos os casos, você precisará de uma [chave de API OpenAI](https://platform.openai.com/account/api-keys). 

Inicializamos um modelo `text-davinci-003`.

In [2]:
# Isto é quando usas o arquivo .env:
from dotenv import load_dotenv
import os
print('Carregando a minha chave Key: ', load_dotenv())
Eddy_API_KEY_OpenAI = os.environ['OPENAI_API_KEY'] 
Eddy_API_KEY_HuggingFace = os.environ["HUGGINGFACEHUB_API_TOKEN"]

Carregando a minha chave Key:  True


In [3]:
from langchain.llms import OpenAI


# Inicializar o Modelo 
openai = OpenAI(
    model_name="text-davinci-003",
    openai_api_key=Eddy_API_KEY_OpenAI
)


<font color="orange">Fazemos uma geração a partir do nosso prompt.</font>

In [4]:
print(openai(prompt))

 Hugging Face (Transformers), OpenAI (openai) e Cohere (cohere).


<font color="orange">Normalmente, `não saberíamos qual é o prompt do usuário de antemão`, então, na verdade, queremos adicioná-lo. Portanto, em vez de escrever o `prompt` diretamente, criamos um `PromptTemplate` com uma única `query` de variável de entrada.</font>

In [5]:
from langchain import PromptTemplate

template = """Responda a pergunta com base no contexto abaixo. Se a pergunta não puder ser respondida
              usando as informações fornecidas, responda com "Não sei".

              
Context: Os Large Language Models (LLMs) são os modelos mais recentes usados em NLP. Seu desempenho
         superior em relação a modelos menores os tornou incrivelmente úteis para desenvolvedores que
         constroem aplicativos habilitados para NLP. Esses modelos podem ser acessados através da
         biblioteca `Transformers` do Hugging Face, via OpenAI usando a biblioteca `openai` e via
         Cohere usando a biblioteca `cohere`.

Question: {query}

Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)



Agora podemos inserir a `query` do usuário no Template de Prompt por meio do parâmetro `query`.

In [6]:
print(
    prompt_template.format(
        query="Quais bibliotecas e provedores de modelos oferecem LLMs?"
    )
)

Responda a pergunta com base no contexto abaixo. Se a pergunta não puder ser respondida
              usando as informações fornecidas, responda com "Não sei".

              
Context: Os Large Language Models (LLMs) são os modelos mais recentes usados em NLP. Seu desempenho
         superior em relação a modelos menores os tornou incrivelmente úteis para desenvolvedores que
         constroem aplicativos habilitados para NLP. Esses modelos podem ser acessados através da
         biblioteca `Transformers` do Hugging Face, via OpenAI usando a biblioteca `openai` e via
         Cohere usando a biblioteca `cohere`.

Question: Quais bibliotecas e provedores de modelos oferecem LLMs?

Answer: 


In [8]:
print(openai(
    prompt_template.format(
        query="Quais bibliotecas e provedores de modelos oferecem LLMs?"
    )
))


 Hugging Face (Transformers), OpenAI (openai) e Cohere (cohere).


<font color="orange">Esta é apenas uma implementação simples, que podemos facilmente substituir por `f-strings` (como `f"insira algum texto personalizado '{custom_text}' etc"`). Mas, usando o objeto `PromptTemplate` do `LangChain`, podemos formalizar o processo, adicionar vários parâmetros e construir os prompts de maneira orientada a objetos.

No entanto, esses não são os únicos benefícios de usar as ferramentas de prompt do `LangChains`.</font>

# Few Shot Prompt Templates

Outro recurso útil oferecido pelo `LangChain` é o objeto `FewShotPromptTemplate`. Isso é ideal para o que chamaríamos de `Aprendizado de poucos tiros` usando nossos prompts.

Para dar algum contexto, as principais fontes de `"conhecimento"` para LLMs são:

* `Conhecimento paramétrico` — o conhecimento foi aprendido durante o treinamento do modelo e é armazenado nos pesos do modelo.

* `Conhecimento da fonte` — o conhecimento é fornecido na entrada do modelo no momento da inferência, ou seja, por meio do `prompt`.

A ideia por trás do `FewShotPromptTemplate` é fornecer `treinamento de poucos tiros como fonte de conhecimento`. Para fazer isso, **adicionamos alguns exemplos aos nossos prompts** que o modelo pode ler e aplicar à entrada do usuário.

# Treinamento de Poucos Tiros (`Few-shot Training`)

<font color="orange">Às vezes, podemos descobrir que um modelo não parece obter o que gostaríamos que fizesse. Podemos ver isso no seguinte exemplo:</font>

In [9]:
prompt = """A seguir, uma conversa com um assistente de IA.
            O assistente é tipicamente sarcástico e espirituoso, produzindo
            respostas engraçadas às perguntas dos usuários. aqui estão alguns exemplos: 

User: Qual é o significado da vida? 
AI: """

openai.temperature = 1.0  # Aumenta a criatividade/aleatoriedade de saída

print(openai(prompt))


 Humm... Bom, eu diria que a resposta para isso está mais para um vinho do que para uma máquina.


<font color="orange">Nesse caso, estamos pedindo algo divertido, uma piada em troca de nossa pergunta séria. Mas obtemos uma resposta séria mesmo com a `temperature` definida para `1.0`. Para ajudar o modelo, podemos dar alguns exemplos do tipo de resposta que gostaríamos:</font>

In [11]:
prompt = """A seguir, uma conversa com um assistente de IA.
            O assistente é tipicamente sarcástico e espirituoso, produzindo
            respostas engraçadas às perguntas dos usuários. aqui estão alguns exemplos: 

User: Como vai você?
AI: Não posso reclamar, mas às vezes ainda o faço.

User: Que horas são?
AI: Então, ... está na hora de comprar um relógio.

User: Qual é o significado da vida?
AI: """

print(openai(prompt))

 42, é claro. Se você quiser entender, vai ter que ler 'O Guia do Mochileiro das Galáxias' novamente.


<font color="pink">Agora obtemos uma resposta muito melhor e fizemos isso por meio do `aprendizado de poucos tiros`, adicionando alguns exemplos por meio de `nosso conhecimento de origem (fonte)`.

Agora, para implementar isso com o `FewShotPromptTemplate` do `LangChain`, precisamos fazer o seguinte:</font>

In [12]:
from langchain import FewShotPromptTemplate


# Criar nossos exemplos
examples = [
    {
        "query": "Como vai você?",
        "answer": "Eu não posso reclamar, mas às vezes eu ainda faço."
    }, {
        "query": "Que horas são?",
        "answer": "Então, está na hora de comprar um relógio."
    }
]

# Criar um exemplo de Template
example_template = """
User: {query}
AI: {answer}
"""

# Crie um exemplo de prompt a partir do Template de acima
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# Agora divida nosso Prompt anterior em um Prefixo e um Sufixo
# O prefixo é nossas instruções
prefix = """A seguir, uma conversa com um assistente de IA.
            O assistente é tipicamente sarcástico e espirituoso, produzindo
            respostas engraçadas às perguntas dos usuários. aqui estão alguns exemplos: 
         """

# e o sufixo é nosso indicador de entrada e saída do usuário
suffix = """
User: {query}
AI: """

# Agora crie o Modelo de Prompt de Poucos Tiros
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

<font color="orange">Agora vamos ver o que isso cria quando alimentamos uma query do usuário...</font>

In [13]:
query = "Qual é o significado da vida?"

print(few_shot_prompt_template.format(query=query))


A seguir, uma conversa com um assistente de IA.
            O assistente é tipicamente sarcástico e espirituoso, produzindo
            respostas engraçadas às perguntas dos usuários. aqui estão alguns exemplos: 
         


User: Como vai você?
AI: Eu não posso reclamar, mas às vezes eu ainda faço.



User: Que horas são?
AI: Então, está na hora de comprar um relógio.



User: Qual é o significado da vida?
AI: 


In [14]:
print(openai(
    few_shot_prompt_template.format(query=query)
            )
     )


 Bem, isso é uma pergunta difícil de responder, mas de acordo com C.S. Lewis, é "ser amado e oferecer amor".


`Mais uma vez, outra boa resposta.`


No entanto, isso é um pouco complicado. Por que passar por todos os itens acima com `FewShotPromptTemplate`, o dicionário de `examples`, etc — quando podemos fazer o mesmo com uma única `f-string`.

Bem, esta abordagem é mais robusta e contém alguns recursos interessantes. Uma delas é a capacidade de `incluir` ou `excluir` exemplos com base no tamanho de nossa query.

Na verdade, isso é muito importante porque o comprimento máximo de nossa saída de `prompt` e geração é `limitado`. Essa limitação é a `janela de contexto máxima` (*max context window*) e é simplesmente o `comprimento do nosso prompt + o comprimento da nossa geração` (que definimos por meio de `max_tokens`).

Portanto, devemos tentar maximizar o número de exemplos que damos ao modelo como `exemplos de aprendizado de poucos tiros`, garantindo que não excedamos a janela de contexto máxima ou aumentemos excessivamente os tempos de processamento.

Vamos ver como funciona a `inclusão/exclusão` dinâmica de exemplos. Primeiro precisamos de mais exemplos: